# Function calling with an OpenAPI specification
> https://cookbook.openai.com/examples/function_calling_with_an_openapi_spec

This notebook is divided into two main sections:
1. How to convert a sample OpenAPI specification into a list of function definitions for the chat completions API.
2. How to use the chat completions API to intelligently invoke these functions based on user instructions.

In [1]:
!pip install -q jsonref # for resolving $ref's in the OpenAPI spec
!pip install -q openai

In [2]:
import os
import json
import jsonref
import requests
from pprint import pp
import openai

In [3]:
from llmcam.fn_to_fc import complete

In [4]:
complete??

Signature: complete(messages, role, content, tools, tool_call_id=None)
Source:   
def complete(messages, role, content, tools, tool_call_id=None):
    "Send completion request with messages, and save the response in messages again"
    messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())
    if res.to_dict().get('tool_calls'):
        complete(messages, role="tool", content=fn_result_content(res), tools=tools, tool_call_id=res.tool_calls[0].id)
    return messages[-1]['role'], messages[-1]['content']
File:      ~/00llmcam/llmcam/fn_to_fc.py
Type:      function

## How to convert a sample OpenAPI specification into a list of function definitions for the chat completions API.

In [5]:
with open('openapi.json.2', 'r') as f:
    oas = jsonref.loads(f.read()) # it's important to load with jsonref, as explained below

In [6]:
for path, methods in oas["paths"].items():
    for method, spec_with_ref in methods.items():
        print(path, ":", method, ":", spec_with_ref['operationId'], ":", spec_with_ref['summary'])

/pet : put : updatePet : Update an existing pet
/pet : post : addPet : Add a new pet to the store
/pet/findByStatus : get : findPetsByStatus : Finds Pets by status
/pet/findByTags : get : findPetsByTags : Finds Pets by tags
/pet/{petId} : get : getPetById : Find pet by ID
/pet/{petId} : post : updatePetWithForm : Updates a pet in the store with form data
/pet/{petId} : delete : deletePet : Deletes a pet
/pet/{petId}/uploadImage : post : uploadFile : uploads an image
/store/inventory : get : getInventory : Returns pet inventories by status
/store/order : post : placeOrder : Place an order for a pet
/store/order/{orderId} : get : getOrderById : Find purchase order by ID
/store/order/{orderId} : delete : deleteOrder : Delete purchase order by ID
/user : post : createUser : Create user
/user/createWithList : post : createUsersWithListInput : Creates list of users with given input array
/user/login : get : loginUser : Logs user into the system
/user/logout : get : logoutUser : Logs out curr

In [7]:
def openapi_to_functions(openapi_spec):
    functions = []

    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. Resolve JSON references.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. Extract a name for the functions.
            function_name = spec.get("operationId")

            # 3. Extract a description and parameters.
            desc = spec.get("description") or spec.get("summary", "")

            schema = {"type": "object", "properties": {}}

            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            functions.append(
                {"type": "function", "function": {"name": function_name, "description": desc, "parameters": schema}}
            )

    return functions


tools = openapi_to_functions(oas)

In [8]:
messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]

In [9]:
complete(messages, "user", "Hi, how many pets are sold?", tools=tools)

TypeError: 'NoneType' object is not callable